In [1]:
function forward_norm2(x, ∂x)
    sum_square = 0
    for i = 1:length(x)
        sum_square += abs2(x[i])
        ∂x[i] *= 2 * x[i]
    end
    norm2 = sqrt(sum_square)
    ∂x = ∂x/norm2/2
    return norm2 , ∂x
end

forward_norm2 (generic function with 1 method)

In [3]:
function backward_norm2(x)
    Nx = length(x)
    η = zeros(Nx+2)
    for i = 1:Nx
        η[i] = abs2(x[i])
        η[Nx+1] += η[i]
    end
    η[Nx+2] = sqrt(η[Nx+1])
    pNx2pNx1 = 1/(2*sqrt(η[Nx+1]))
    pNx1pNi = ones(Nx,1)
    pNx2pNi = pNx2pNx1.*pNx1pNi
    pNipxi = zeros(Nx,1)
    for i = 1:Nx
        pNipxi[i] = 2*x[i]
    end
    pNx2pxi = pNx2pNi .* pNipxi
    return η[Nx+2],pNx2pxi
end

backward_norm2 (generic function with 1 method)

$\eta$ contains two parts: the first Nx $\eta$ is the square of the elements $x_i$, while the next $\eta$, namely $\eta$[Nx+1] is the sum of the square, and the last is the suqare root of $\eta$[Nx+1].

In [4]:
function poorman_norm(x::Vector{<:Real})
	nm2 = zero(real(eltype(x)))
	for i=1:length(x)
		nm2 += abs2(x[i])
	end
	ret = sqrt(nm2)
	return ret
end

poorman_norm (generic function with 1 method)

In [5]:
using ForwardDiff
using Test

In [6]:
@testset "Forward AD" begin
    for i in 1:100
        N = rand(1:100)
        x = rand(N)
        ∂x = ones(N)
        norm2, px_2 = forward_norm2(x, ∂x)
        @test norm2 ≈ poorman_norm(x)
        @test px_2 ≈ ForwardDiff.gradient(poorman_norm, x)        
    end
end

Test Summary: | Pass  Total  Time
Forward AD    |  200    200  4.4s


Test.DefaultTestSet("Forward AD", Any[], 200, false, false, true, 1.683528609703038e9, 1.683528614124185e9, false)

In [11]:
@testset "Backward AD"  begin
    for i in 1:100
        N = rand(1:100)
        x = rand(N)
        norm2,px_2 = backward_norm2(x)
        @test norm2 ≈ poorman_norm(x)
        @test px_2 ≈ ForwardDiff.gradient(poorman_norm, x)
    end       
end

Test Summary: | Pass  Total  Time
Backward AD   |  200    200  0.0s


Test.DefaultTestSet("Backward AD", Any[], 200, false, false, true, 1.683528675511213e9, 1.683528675524039e9, false)

for an N-d x, N+1 intermediate states are cashed in my reverse mode AD